# Model Runs per Half Inning Using Linear Regression

&#128308; Rough Draft

A linear regression model is fitted to see how much each type of event, such as a single or a home run, contributes to scoring in a half-inning.

Linear regression models are good explanatory models.  A more complicated model would likely predict better, but be harder to interpret.

In Sabermetric terminology, the linear regression coefficients are called "linear weights".

The first step is to create a DataFrame that has offensive stats per half inning.  As the output of the cwevent parser is missing a few necessary fields, I added these fields by parsing the Retrosheet event file.  These new fields were verified to be 100% consistent with those created by the cwgame parser.

## Baseball Assumptions
* The goal of each offensive half-inning is to score as many runs as possible
* The goal of each defensive half-inning is to allow as few runs as possible
  
The above assumptions are not entirely correct for the home half of the 9th or later innings in which the offensive team is trying to end the game with a win, and the defensive team is trying to prevent the game from ending with a loss.

As such, one linear model is created for all innings and another will be created that only uses the first 8 innings.

Rain-shortened games may have less than 8 complete innings, however it is difficult for either team to predict when an umpire will call a game.  Arguably, the goal of scoring the most runs and allowing the least runs, applies to all innings in a rain-shortened game.

## Fields Added to Output of cwevent

The output of cwevent is described at: http://chadwick.sourceforge.net/doc/cwevent.html

cwevent does not produce a field for: so, sb, cs, bk, bb, ibb, hbp, and xi.  A boolean column was extracted by parsing event_tx to create these when retrosheet_collect.py was run as part of the parsing, collecting, and wrangling the data produced by the cw parsers.

cwevent has h_cd with values of 1, 2, 3, 4 for single, double, triple, home run, but for ease of processing, a boolean flag was created for each.  When the event data is grouped by half-inning, the boolean values can be summed to get the total number of singles, doubles, triples, and home runs per half-inning.

As cwevent has been used and tested for years, I will not attempt to rewrite it. Instead I am adding a few flags to the output of cwevent by parsing event_tx, and then verifying that when aggregated, these exactly match the results of the cwgame parser.

In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
import re

In [2]:
import sys

# import data_helper.py from download_scripts directory
sys.path.append('../download_scripts')
import data_helper as dh

In [3]:
# Linear Regression modules
import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrices

In [4]:
data_dir = Path('../data')
lahman_data = data_dir.joinpath('lahman/wrangled').resolve()
retrosheet_data = data_dir.joinpath('retrosheet/wrangled').resolve()

## Read in the Data

In [5]:
event = dh.from_csv_with_types(retrosheet_data / 'event.csv.gz')

In [6]:
pd.set_option('display.max_columns', 50)
event.head(5)

,game_id,inn_ct,home_half,away_score_ct,home_score_ct,bat_id,pit_id,event_tx,h_cd,outs,e,event_id,team_id,opponent_team_id,inn_runs_ct,start_bases_cd,end_bases_cd,r,fate_runs_ct,ab,sh,sf,dp,tp,wp,pb,inn_end,pa,bat_safe_err,so,sb,cs,bk,ibb,bb,hbp,xi,single,double,triple,hr,h
0,BAL195504120,1,0,0,0,goodb101,colej101,S,1,0,0,1,BOS,BAL,0,0,1,0,1,True,False,False,False,False,False,False,False,True,False,False,0,0,False,False,False,False,False,True,False,False,False,True
1,BAL195504120,1,0,0,0,joose101,colej101,S7.1-3,1,0,0,2,BOS,BAL,0,1,5,0,1,True,False,False,False,False,False,False,False,True,False,False,0,0,False,False,False,False,False,True,False,False,False,True
2,BAL195504120,1,0,0,0,throf101,colej101,64(1)/FO.3-H,0,1,0,3,BOS,BAL,0,5,1,1,0,True,False,False,False,False,False,False,False,True,False,False,0,0,False,False,False,False,False,False,False,False,False,False
3,BAL195504120,1,0,1,0,jensj101,colej101,36(1)/FO,0,1,0,4,BOS,BAL,1,1,1,0,0,True,False,False,False,False,False,False,False,True,False,False,0,0,False,False,False,False,False,False,False,False,False,False
4,BAL195504120,1,0,1,0,whits103,colej101,4/L,0,1,0,5,BOS,BAL,1,1,1,0,0,True,False,False,False,False,False,False,True,True,False,False,0,0,False,False,False,False,False,False,False,False,False,False


In [7]:
# most processing is done by year, add a year column
event['year'] = event['game_id'].str[3:7].astype('int')

In [8]:
# for now, just consider half-innings in the last 20 seasons
event_inn_all = event.query('year >= 2000')
event_inn_8 = event.query('inn_ct < 9 and year >= 2000')

In [9]:
# columns to aggregate per half-inning
agg_cols = ['pb', 'wp', 'dp', 'bb', 'outs', 'so', 'hbp', 'triple', 'single', 'tp', 'sf',
            'r', 'double', 'pa', 'bat_safe_err', 'h', 'sb', 'sh', 'ibb', 'ab', 'cs',
            'hr', 'xi', 'e', 'bk']

In [10]:
# group by half-inning
key = ['game_id', 'inn_ct', 'home_half']
inn_all = event_inn_all[key + agg_cols].groupby(key).agg('sum')
inn_8 = event_inn_8[key + agg_cols].groupby(key).agg('sum')

In [11]:
inn_all.head(18)

pb   wp   dp   bb  outs   so  hbp  triple  \
game_id      inn_ct home_half                                               
ANA200004030 1      0          0.0  0.0  0.0  1.0     3  0.0  0.0     0.0   
                    1          0.0  0.0  1.0  0.0     3  1.0  0.0     0.0   
             2      0          0.0  0.0  0.0  1.0     3  0.0  0.0     0.0   
                    1          0.0  0.0  0.0  1.0     3  0.0  0.0     0.0   
             3      0          0.0  0.0  0.0  0.0     3  0.0  0.0     0.0   
                    1          0.0  0.0  0.0  0.0     3  1.0  0.0     0.0   
             4      0          0.0  0.0  0.0  0.0     3  1.0  0.0     0.0   
                    1          0.0  0.0  0.0  0.0     3  0.0  0.0     0.0   
             5      0          0.0  0.0  0.0  1.0     3  1.0  0.0     0.0   
                    1          0.0  0.0  0.0  0.0     3  1.0  0.0     0.0   
             6      0          0.0  0.0  0.0  0.0     3  0.0  0.0     0.0   
                    1          0.0  0.0  0.0  2.0     3  1.0  0.0     0.0   
             7      0          0.0  0.0  0.0  0.0     3  0.0  0.0     0.0   
                    1          0.0  0.0  0.0  0.0     3  0.0  0.0     0.0   
             8      0          0.0  0.0  0.0  0.0     3  1.0  0.0     0.0   
                    1          0.0  0.0  0.0  1.0     3  1.0  0.0     0.0   
             9      0          0.0  0.0  0.0  0.0     3  0.0  0.0     0.0   
                    1          0.0  0.0  0.0  1.0     3  1.0  0.0     0.0   

                               single   tp   sf  r  double   pa  bat_safe_err  \
game_id      inn_ct home_half                                                   
ANA200004030 1      0             1.0  0.0  0.0  0     0.0  4.0           0.0   
                    1             1.0  0.0  0.0  0     0.0  3.0           0.0   
             2      0             0.0  0.0  0.0  0     0.0  4.0           0.0   
                    1             1.0  0.0  0.0  1     0.0  5.0           0.0   
             3      0             0.0  0.0  0.0  0     0.0  3.0           0.0   
                    1             0.0  0.0  0.0  0     1.0  4.0           0.0   
             4      0             1.0  0.0  0.0  0     0.0  3.0           0.0   
                    1             0.0  0.0  0.0  0     0.0  3.0           0.0   
             5      0             0.0  0.0  0.0  0     0.0  5.0           1.0   
                    1             3.0  0.0  0.0  0     0.0  6.0           0.0   
             6      0             2.0  0.0  0.0  2     0.0  6.0           0.0   
                    1             0.0  0.0  0.0  0     0.0  5.0           0.0   
             7      0             0.0  0.0  0.0  1     0.0  4.0           0.0   
                    1             1.0  0.0  0.0  0     0.0  4.0           0.0   
             8      0             0.0  0.0  0.0  0     0.0  3.0           0.0   
                    1             0.0  0.0  0.0  0     0.0  4.0           0.0   
             9      0             0.0  0.0  0.0  0     0.0  3.0           0.0   
                    1             2.0  0.0  0.0  1     0.0  6.0           0.0   

                                 h  sb   sh  ibb   ab  cs   hr   xi  e   bk  
game_id      inn_ct home_half                                                
ANA200004030 1      0          1.0   0  0.0  0.0  3.0   1  0.0  0.0  0  0.0  
                    1          1.0   0  0.0  0.0  3.0   0  0.0  0.0  0  0.0  
             2      0          0.0   0  0.0  0.0  3.0   0  0.0  0.0  0  0.0  
                    1          2.0   0  0.0  0.0  4.0   1  1.0  0.0  0  0.0  
             3      0          0.0   0  0.0  0.0  3.0   0  0.0  0.0  0  0.0  
                    1          1.0   0  0.0  0.0  4.0   0  0.0  0.0  0  0.0  
             4      0          1.0   0  0.0  0.0  3.0   1  0.0  0.0  0  0.0  
                    1          0.0   0  0.0  0.0  3.0   0  0.0  0.0  0  0.0  
             5      0          0.0   0  0.0  0.0  4.0   0  0.0  0.0  1  0.0  
            

In [12]:
# spot check that the score was 3-2, as seen above
usecols = ['game_id', 'bat_last', 'team_id', 'opponent_team_id', 'r', 'h', 'e',
           'lob', 'line_tx', 'ab', 'double', 'triple', 'hr', 'rbi', 'sh', 'sf',
           'hbp', 'bb', 'ibb', 'so', 'sb', 'cs', 'gidp', 'xi', 'er', 'ter', 'wp',
           'bk', 'po', 'a', 'pb', 'dp', 'tp', 'year']
team_game = dh.from_csv_with_types(retrosheet_data / 'team_game.csv.gz', usecols=usecols)
team_game.query('game_id == "ANA200004030"')

,game_id,bat_last,team_id,opponent_team_id,r,h,e,lob,line_tx,ab,double,triple,hr,rbi,sh,sf,hbp,bb,ibb,so,sb,cs,gidp,xi,er,ter,wp,bk,po,a,pb,dp,tp,year
167420,ANA200004030,True,ANA,NYA,2,10,1,11,010000001,35,1,0,1,2,0,0,0,5,0,6,0,1,0,0,3,3,0,0,27,16,0,0,0,2000
167421,ANA200004030,False,NYA,ANA,3,6,0,5,000002100,32,0,0,2,3,0,0,0,3,0,3,0,2,0,0,2,2,0,0,27,5,0,1,0,2000


In [13]:
# bb (base on balls) is defined to include ibb (intentional base on ball)
# create a new column for unintentional base on balls
inn_all['ubb'] = inn_all['bb'] - inn_all['ibb']
inn_8['ubb'] = inn_8['bb'] - inn_8['ibb']

In [14]:
inn_all['r'].agg(['mean', 'std'])

mean    0.510788
std     1.034444
Name: r, dtype: float64

In [15]:
inn_8['r'].agg(['mean', 'std'])

mean    0.519664
std     1.046788
Name: r, dtype: float64

Although the above values are close, it is reasonable to expect that the average number of runs per inning will be lower when including all innings vs when including just the first 8 innings.

After the 8th inning, the strategy is not to maximum runs but to end the game with a win.

In [16]:
# perhaps the easiest way to specify the dependent variables is to use R notation
# note that -1 means to not fit a y-intercept.
formula = 'r ~ ubb + e + bat_safe_err + hbp + single + double + dp +triple +hr +outs +sf +pb +wp +cs -1'
model_all = smf.ols(formula=formula, data=inn_all)
result_all = model_all.fit()
print(result_all.summary())

                                 OLS Regression Results                                
Dep. Variable:                      r   R-squared (uncentered):                   0.826
Model:                            OLS   Adj. R-squared (uncentered):              0.826
Method:                 Least Squares   F-statistic:                          2.943e+05
Date:                Mon, 03 Feb 2020   Prob (F-statistic):                        0.00
Time:                        20:06:44   Log-Likelihood:                     -5.9901e+05
No. Observations:              870031   AIC:                                  1.198e+06
Df Residuals:                  870017   BIC:                                  1.198e+06
Df Model:                          14                                                  
Covariance Type:            nonrobust                                                  
                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------

In [17]:
# number of observations (number of half-innings)
result_all.nobs

870031.0

In [18]:
# adjusted R^2 (percent of variance explained)
result_all.rsquared_adj

0.8256677157092751

In [19]:
# coefficients
result_all.params

ubb             0.335461
e               0.355507
bat_safe_err    0.165494
hbp             0.368815
single          0.482651
double          0.731579
dp             -0.285470
triple          0.962615
hr              1.389602
outs           -0.094324
sf              0.540088
pb              0.170464
wp              0.166710
cs             -0.217145
dtype: float64

In [20]:
# 95% confidence interval
result_all.conf_int()

,0,1
ubb,0.333745,0.337178
e,0.349598,0.361416
bat_safe_err,0.157626,0.173361
hbp,0.363750,0.373881
single,0.481445,0.483858
double,0.729342,0.733815
dp,-0.288889,-0.282050
triple,0.955570,0.969660
hr,1.386749,1.392455
outs,-0.094805,-0.093843


## Examine the Model for 8 Complete Innings

In [21]:
model_8 = smf.ols(formula=formula, data=inn_8)
result_8 = model_8.fit()
print(result_8.summary())

                                 OLS Regression Results                                
Dep. Variable:                      r   R-squared (uncentered):                   0.828
Model:                            OLS   Adj. R-squared (uncentered):              0.828
Method:                 Least Squares   F-statistic:                          2.672e+05
Date:                Mon, 03 Feb 2020   Prob (F-statistic):                        0.00
Time:                        20:06:45   Log-Likelihood:                     -5.3968e+05
No. Observations:              776987   AIC:                                  1.079e+06
Df Residuals:                  776973   BIC:                                  1.080e+06
Df Model:                          14                                                  
Covariance Type:            nonrobust                                                  
                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------

In [22]:
# number of observations (number of half-innings)
result_8.nobs

776987.0

In [23]:
# adjusted R^2 (percent of variance explained)
result_8.rsquared_adj

0.8280340336878028

In [24]:
result_8.rsquared_adj - result_all.rsquared_adj

0.00236631797852771

The R^2 value is very slightly higher.  A higher value might be expected given that game strategy is the same over the first 8 innings.

In [25]:
# coefficients
result_8.params

ubb             0.339076
e               0.355425
bat_safe_err    0.169754
hbp             0.373747
single          0.486960
double          0.733647
dp             -0.288173
triple          0.964403
hr              1.387851
outs           -0.095541
sf              0.535517
pb              0.170134
wp              0.170515
cs             -0.218926
dtype: float64

In [26]:
# 95% confidence interval
result_8.conf_int()

,0,1
ubb,0.337249,0.340903
e,0.349203,0.361646
bat_safe_err,0.161429,0.178080
hbp,0.368371,0.379124
single,0.485680,0.488241
double,0.731290,0.736004
dp,-0.291781,-0.284564
triple,0.957007,0.971799
hr,1.384837,1.390865
outs,-0.096054,-0.095027


In [27]:
# add a column for all_innings coefficients
ci = result_8.conf_int()
ci['param_all'] = result_all.params
ci

,0,1,param_all
ubb,0.337249,0.340903,0.335461
e,0.349203,0.361646,0.355507
bat_safe_err,0.161429,0.178080,0.165494
hbp,0.368371,0.379124,0.368815
single,0.485680,0.488241,0.482651
double,0.731290,0.736004,0.731579
dp,-0.291781,-0.284564,-0.285470
triple,0.957007,0.971799,0.962615
hr,1.384837,1.390865,1.389602
outs,-0.096054,-0.095027,-0.094324


In [28]:
# are the coefficients using all innings within the CI of the model using just 8 innings?
ci.apply(lambda row: row[0] <= row['param_all'] <= row[1], axis=1)

ubb             False
e                True
bat_safe_err     True
hbp              True
single          False
double           True
dp               True
triple           True
hr               True
outs            False
sf               True
pb               True
wp               True
cs               True
dtype: bool

Three of the coefficients for model_all are not always within the CI for model_8.  By chance, only about 1 of the coefficients should be outside the 95% CI.  This suggests that run scoring is slightly different after the 8th inning.

# Summary
Discuss relative values of home run, triple, double and single.

In [33]:
single = result_8.params['single']
double = result_8.params['double']
triple = result_8.params['triple']
hr = result_8.params['hr']

In [39]:
# Slugging Percent uses 4:3:2:1
print(f'{hr:4.2f}:{triple:4.2f}:{double:4.2f}:{single:4.2f}')

1.39:0.96:0.73:0.49
